<a href="https://colab.research.google.com/github/ShotgunkeeperVZ/detectron2/blob/main/Copy_of_Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install -U albumentations


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

DATASET = "/content/gdrive/MyDrive/Dataset/"

In [ ]:
import albumentations as A
import cv2
import numpy as np
import pandas as pd
import os

import random

from matplotlib import pyplot as plt


def visualize(image,bbox=None):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    
    if(bbox != None):
        for box in bbox:
            x = int(box[0])
            y = int(box[1])
            w = int(box[2])
            h = int(box[3])
            
            # print(x,y,w,h)
            cv2.rectangle(image,(x,y),(x+w,y+h),color=(255,0,0),thickness=6)
            

        
    plt.imshow(image)
    plt.show()


ANNOTATIONS_CSV_PATH = DATASET + "Annotations.csv"
AUGMENTED_CSV_PATH = DATASET + "augment_labels.csv"
RAW_DATA = DATASET + "Data/Train/"
AUGMENT_DIR = DATASET + "Data_Augmented/"

# Probabilities


annotations_df = pd.read_csv(ANNOTATIONS_CSV_PATH)

if(os.path.isfile(AUGMENTED_CSV_PATH) == False):
  os.system("rm -rf {}*".format(AUGMENT_DIR))
  transform = A.Compose([
      A.LongestMaxSize(p=0.9999,max_size=800),
      A.OneOf(transforms=(
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.5)
   ),p=1.0,always_apply=True),


      A.Affine(translate_percent={"x":(0,0.5),"y":(0.2,0.75)},p=0.5),
      A.ChannelShuffle(p=0.25),
      A.Posterize (num_bits=4, always_apply=False, p=0.3),
      A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.6),
      A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.35),

      A.BBoxSafeRandomCrop(p=0.6),
      A.OneOf(transforms=(A.Affine(scale=(1.15, 0.9)),
              A.Affine(scale=(0.92, 1.05))), p=0.7),
      A.RandomCropFromBorders(crop_left=0.15, crop_right=0.15, crop_top=0.15, crop_bottom=0.15, always_apply=False, p=0.35),
      A.OneOf(transforms=(
          A.Affine(shear=(2,0),p=1,scale=0.9),
          A.Affine(shear=(0,3),p=1,scale=0.9)
      ),p=0.67),
      A.RandomResizedCrop (2000, 1000, scale=(0.7, 1.1), ratio=(0.8, 1.2), interpolation=1, always_apply=False, p=0.65),
      A.RandomBrightness(limit=(-0.1, 0.1),p=1
                        ),
      A.RandomRotate90(p=0.45),
      A.Rotate(limit=360, interpolation=1, border_mode=2, value=None, mask_value=None,
              rotate_method='largest_box', crop_border=True, always_apply=False, p=0.2),
      A.Flip(p=0.3),
      A.RandomScale(p=0.8),
      
  ], bbox_params=A.BboxParams(format='coco', min_visibility=0.1))


  for image in os.listdir(RAW_DATA):
      relative_path = os.path.join(RAW_DATA, image)
      if os.path.isfile(relative_path):
          image_name = image
          absolute_path = os.path.abspath(relative_path)

          height = annotations_df[annotations_df.image_name == image].head(1).image_height
          width = annotations_df[annotations_df.image_name == image].head(1).image_width

          annotations = pd.DataFrame()

          image_annotations = annotations_df[annotations_df.image_name == image]
          image_annotations["label"] = "SPEC"
          image = cv2.imread(absolute_path)
          # cv2.imwrite(AUGMENT_DIR+image_name,image)
          # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

          bboxes = (image_annotations[[
                    "bbox_x", "bbox_y", "bbox_width", "bbox_height", "label"]].to_numpy().tolist())
          
          for i in range(3 * int(np.random.normal(loc=3,scale=2))):
              transformed = transform(image=image, bboxes=bboxes)
              if(transformed["bboxes"] == []):
                continue
              # visualize(transformed["image"],transformed["bboxes"])
              
              image_name_local = image_name.split(".")[0]+"_A"+str(i)+".jpg"
              for box in transformed["bboxes"]:
                  x = int(box[0])
                  y = int(box[1])
                  w = int(box[2])
                  h = int(box[3])
                  new_box = pd.Series({"label_name":"Spec","bbox_x":x,"bbox_y":y,"bbox_width":w,"bbox_height":h,
                                      "image_name":image_name_local,"image_width":transformed["image"].shape[1],"image_height":transformed["image"].shape[0]})
                  annotations_df = pd.concat([annotations_df,new_box.to_frame().T],ignore_index=True)
              transformed_image = cv2.cvtColor(transformed["image"], cv2.COLOR_BGR2RGB)
              cv2.imwrite(AUGMENT_DIR + image_name_local,transformed_image)
              image_name_local = ""
  annotations_df.to_csv(index=False,path_or_buf=DATASET+"augment_labels.csv")

In [ ]:
import numpy as np
import pandas as pd
# import torch
import os
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog

ANNOTATIONS_CSV_PATH = DATASET+"augment_labels.csv"
TRAIN_DATA_DIR = DATASET+"Data_Augmented/"
TEST_DATA_DIR = DATASET+"Data/Test/"

annotations_df = pd.read_csv(ANNOTATIONS_CSV_PATH)




def register_dataset(directory):
    dataset = []
    for image in os.listdir(directory):

        relative_path = os.path.join(directory, image)
        if os.path.isfile(relative_path):
            absolute_path = os.path.abspath(relative_path)
            height = annotations_df[annotations_df.image_name == image]["image_height"].tolist()[0]
            width = annotations_df[annotations_df.image_name == image]["image_width"].tolist()[0]
            # display(height,width)
            print(image,height,width)
            annotations = pd.DataFrame()
            image_annotations = annotations_df[annotations_df.image_name == image]
           

            annotations["bbox"] = image_annotations[["bbox_x", "bbox_y","bbox_width", "bbox_height"]].to_numpy().tolist()
            annotations["bbox"] = annotations["bbox"].apply(
                lambda list: [float(i) for i in list])

            annotations["bbox_mode"] = BoxMode.XYWH_ABS
            annotations["category_id"] = 0
            
            annotations = list(annotations.to_dict('index').values())
            
            dataset.append({
                "file_name":absolute_path,
                "height":height,
                "width" : width,
                "image_id":image + "_id",
                "annotations":annotations
            })
            # print(dataset)
    return dataset   
register_dataset(TRAIN_DATA_DIR)

DatasetCatalog.register("Spec_Train", lambda d=None: register_dataset(TRAIN_DATA_DIR))
MetadataCatalog.get("Spec_Train").set(thing_classes=["spec"])
# DatasetCatalog.register("Spec_Test", lambda d=None: register_dataset(TEST_DATA_DIR))
# MetadataCatalog.get("Spec_Test").set(thing_classes=["spec"])


spec_dataset = MetadataCatalog.get("Spec_Train")



In [ ]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
os.makedirs(DATASET + "OUTPUT", exist_ok=True)

from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("Spec_Train",)
# cfg.DATASETS.TEST = ("Spec_Test",)

cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.IMS_PER_BATCH = 4  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.0003  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = DATASET + "OUTPUT"

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()


In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow
for image in os.listdir(TEST_DATA_DIR):
  im = cv2.imread(TEST_DATA_DIR + image)
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  v = Visualizer(im[:, :, ::-1],metadata=spec_dataset,scale=0.5)
  out = v.draw_instance_predictions(outputs["instances"])
  cv2_imshow(out.get_image()[:, :, ::-1])